# Project Title

**Authors:** Student 1, Student 2, Student 3
***

## Overview

A one-paragraph overview of the project, including the business problem, data, methods, results and recommendations.

## Business Problem

Summary of the business problem you are trying to solve, and the data questions that you plan to answer to solve them.

***
Questions to consider:
* What are the business's pain points related to this project?
* How did you pick the data analysis question(s) that you did?
* Why are these questions important from a business perspective?
***

## Data Understanding

Describe the data being used for this project.
***
Questions to consider:
* Where did the data come from, and how do they relate to the data analysis questions?
* What do the data represent? Who is in the sample and what variables are included?
* What is the target variable?
* What are the properties of the variables you intend to use?
***

In [44]:
import pandas as pd

from surprise import Dataset, Reader, accuracy
from surprise.model_selection import cross_validate,GridSearchCV
from surprise.prediction_algorithms import SVD, SVDpp, NMF, BaselineOnly,\
                                            NormalPredictor, KNNBasic
from sklearn.model_selection import train_test_split
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt

## EDA

In [2]:
movies = pd.read_csv('./data/movies.csv')
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [3]:
movies.isna().sum()

movieId    0
title      0
genres     0
dtype: int64

In [4]:
movies.title.duplicated().sum()

5

In [5]:
movies.drop_duplicates(subset='title', inplace=True)

In [6]:
movies.title.duplicated().sum()

0

In [7]:
movies.shape

(9737, 3)

In [8]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [9]:
movies[['title', 'year']] = movies.title.str.split('(', n=1, expand=True)
movies

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995)
1,2,Jumanji,Adventure|Children|Fantasy,1995)
2,3,Grumpier Old Men,Comedy|Romance,1995)
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995)
4,5,Father of the Bride Part II,Comedy,1995)
...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic,Action|Animation|Comedy|Fantasy,2017)
9738,193583,No Game No Life: Zero,Animation|Comedy|Fantasy,2017)
9739,193585,Flint,Drama,2017)
9740,193587,Bungo Stray Dogs: Dead Apple,Action|Animation,2018)


In [10]:
movies['year'] = movies.year.str.replace(')', '')
movies

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995
...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic,Action|Animation|Comedy|Fantasy,2017
9738,193583,No Game No Life: Zero,Animation|Comedy|Fantasy,2017
9739,193585,Flint,Drama,2017
9740,193587,Bungo Stray Dogs: Dead Apple,Action|Animation,2018


In [11]:
movies = movies[['movieId', 'year', 'title', 'genres']]
movies

,movieId,year,title,genres
0,1,1995,Toy Story,Adventure|Animation|Children|Comedy|Fantasy
1,2,1995,Jumanji,Adventure|Children|Fantasy
2,3,1995,Grumpier Old Men,Comedy|Romance
3,4,1995,Waiting to Exhale,Comedy|Drama|Romance
4,5,1995,Father of the Bride Part II,Comedy
...,...,...,...,...
9737,193581,2017,Black Butler: Book of the Atlantic,Action|Animation|Comedy|Fantasy
9738,193583,2017,No Game No Life: Zero,Animation|Comedy|Fantasy
9739,193585,2017,Flint,Drama
9740,193587,2018,Bungo Stray Dogs: Dead Apple,Action|Animation


In [12]:
movies['genres']=[row.strip().lower().replace('|',', ') for row in movies['genres']]
movies

,movieId,year,title,genres
0,1,1995,Toy Story,"adventure, animation, children, comedy, fantasy"
1,2,1995,Jumanji,"adventure, children, fantasy"
2,3,1995,Grumpier Old Men,"comedy, romance"
3,4,1995,Waiting to Exhale,"comedy, drama, romance"
4,5,1995,Father of the Bride Part II,comedy
...,...,...,...,...
9737,193581,2017,Black Butler: Book of the Atlantic,"action, animation, comedy, fantasy"
9738,193583,2017,No Game No Life: Zero,"animation, comedy, fantasy"
9739,193585,2017,Flint,drama
9740,193587,2018,Bungo Stray Dogs: Dead Apple,"action, animation"


In [13]:
rating = pd.read_csv('./data/ratings.csv')
rating

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [14]:
rating['timestamp'] = pd.to_datetime(rating['timestamp'], unit='s')
rating

,userId,movieId,rating,timestamp
0,1,1,4.0,2000-07-30 18:45:03
1,1,3,4.0,2000-07-30 18:20:47
2,1,6,4.0,2000-07-30 18:37:04
3,1,47,5.0,2000-07-30 19:03:35
4,1,50,5.0,2000-07-30 18:48:51
...,...,...,...,...
100831,610,166534,4.0,2017-05-03 21:53:22
100832,610,168248,5.0,2017-05-03 22:21:31
100833,610,168250,5.0,2017-05-08 19:50:47
100834,610,168252,5.0,2017-05-03 21:19:12


In [15]:
rating.isna().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [16]:
rating.duplicated().sum()

0

In [17]:
rating['rating'].value_counts()

4.0    26818
3.0    20047
5.0    13211
3.5    13136
4.5     8551
2.0     7551
2.5     5550
1.0     2811
1.5     1791
0.5     1370
Name: rating, dtype: int64

In [18]:
rating['rating'].value_counts(normalize=True)

4.0    0.265957
3.0    0.198808
5.0    0.131015
3.5    0.130271
4.5    0.084801
2.0    0.074884
2.5    0.055040
1.0    0.027877
1.5    0.017762
0.5    0.013586
Name: rating, dtype: float64

In [19]:
rating.shape

(100836, 4)

In [20]:
movie_rating = movies.merge(rating, on='movieId', how='outer')
movie_rating

,movieId,year,title,genres,userId,rating,timestamp
0,1,1995,Toy Story,"adventure, animation, children, comedy, fantasy",1.0,4.0,2000-07-30 18:45:03
1,1,1995,Toy Story,"adventure, animation, children, comedy, fantasy",5.0,4.0,1996-11-08 06:36:02
2,1,1995,Toy Story,"adventure, animation, children, comedy, fantasy",7.0,4.5,2005-01-25 06:52:26
3,1,1995,Toy Story,"adventure, animation, children, comedy, fantasy",15.0,2.5,2017-11-13 12:59:30
4,1,1995,Toy Story,"adventure, animation, children, comedy, fantasy",17.0,4.5,2011-05-18 05:28:03
...,...,...,...,...,...,...,...
100849,64997,NaN,NaN,NaN,68.0,2.5,2008-12-28 20:55:15
100850,144606,NaN,NaN,NaN,111.0,4.0,2018-01-31 23:27:37
100851,147002,NaN,NaN,NaN,318.0,4.0,2017-08-08 15:45:52
100852,26958,NaN,NaN,NaN,509.0,3.5,2015-07-04 17:42:33


In [21]:
movie_rating.isna().sum()

movieId       0
year         23
title         6
genres        6
userId       18
rating       18
timestamp    18
dtype: int64

In [22]:
movie_rating.dropna(inplace=True)

In [23]:
movie_rating

,movieId,year,title,genres,userId,rating,timestamp
0,1,1995,Toy Story,"adventure, animation, children, comedy, fantasy",1.0,4.0,2000-07-30 18:45:03
1,1,1995,Toy Story,"adventure, animation, children, comedy, fantasy",5.0,4.0,1996-11-08 06:36:02
2,1,1995,Toy Story,"adventure, animation, children, comedy, fantasy",7.0,4.5,2005-01-25 06:52:26
3,1,1995,Toy Story,"adventure, animation, children, comedy, fantasy",15.0,2.5,2017-11-13 12:59:30
4,1,1995,Toy Story,"adventure, animation, children, comedy, fantasy",17.0,4.5,2011-05-18 05:28:03
...,...,...,...,...,...,...,...
100843,193581,2017,Black Butler: Book of the Atlantic,"action, animation, comedy, fantasy",184.0,4.0,2018-09-16 14:44:42
100844,193583,2017,No Game No Life: Zero,"animation, comedy, fantasy",184.0,3.5,2018-09-16 14:52:25
100845,193585,2017,Flint,drama,184.0,3.5,2018-09-16 14:56:45
100846,193587,2018,Bungo Stray Dogs: Dead Apple,"action, animation",184.0,3.5,2018-09-16 15:00:21


In [24]:
links = pd.read_csv('./data/links.csv')
links

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


In [25]:
tags = pd.read_csv('./data/tags.csv')
tags 

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


In [26]:
final_movie_rating = movie_rating.merge(tags, on='movieId', how='outer')
final_movie_rating

,movieId,year,title,genres,userId_x,rating,timestamp_x,userId_y,tag,timestamp_y
0,1,1995,Toy Story,"adventure, animation, children, comedy, fantasy",1.0,4.0,2000-07-30 18:45:03,336.0,pixar,1.139046e+09
1,1,1995,Toy Story,"adventure, animation, children, comedy, fantasy",1.0,4.0,2000-07-30 18:45:03,474.0,pixar,1.137207e+09
2,1,1995,Toy Story,"adventure, animation, children, comedy, fantasy",1.0,4.0,2000-07-30 18:45:03,567.0,fun,1.525286e+09
3,1,1995,Toy Story,"adventure, animation, children, comedy, fantasy",5.0,4.0,1996-11-08 06:36:02,336.0,pixar,1.139046e+09
4,1,1995,Toy Story,"adventure, animation, children, comedy, fantasy",5.0,4.0,1996-11-08 06:36:02,474.0,pixar,1.137207e+09
...,...,...,...,...,...,...,...,...,...,...
285756,34482,NaN,NaN,NaN,NaN,NaN,NaT,474.0,In Netflix queue,1.137202e+09
285757,85565,NaN,NaN,NaN,NaN,NaN,NaT,543.0,Comedy,1.377022e+09
285758,156605,NaN,NaN,NaN,NaN,NaN,NaT,567.0,quirky,1.525283e+09
285759,156605,NaN,NaN,NaN,NaN,NaN,NaT,567.0,sweet,1.525283e+09


In [27]:
final_movie_rating.isna().sum()

movieId            0
year              24
title             24
genres            24
userId_x          24
rating            24
timestamp_x       24
userId_y       52527
tag            52527
timestamp_y    52527
dtype: int64

In [28]:
final_movie_rating.dropna(inplace=True)
final_movie_rating

,movieId,year,title,genres,userId_x,rating,timestamp_x,userId_y,tag,timestamp_y
0,1,1995,Toy Story,"adventure, animation, children, comedy, fantasy",1.0,4.0,2000-07-30 18:45:03,336.0,pixar,1.139046e+09
1,1,1995,Toy Story,"adventure, animation, children, comedy, fantasy",1.0,4.0,2000-07-30 18:45:03,474.0,pixar,1.137207e+09
2,1,1995,Toy Story,"adventure, animation, children, comedy, fantasy",1.0,4.0,2000-07-30 18:45:03,567.0,fun,1.525286e+09
3,1,1995,Toy Story,"adventure, animation, children, comedy, fantasy",5.0,4.0,1996-11-08 06:36:02,336.0,pixar,1.139046e+09
4,1,1995,Toy Story,"adventure, animation, children, comedy, fantasy",5.0,4.0,1996-11-08 06:36:02,474.0,pixar,1.137207e+09
...,...,...,...,...,...,...,...,...,...,...
285699,187595,2018,Solo: A Star Wars Story,"action, adventure, children, sci-fi",586.0,5.0,2018-06-25 04:05:56,62.0,star wars,1.528935e+09
285724,193565,2010,Gintama: The Movie,"action, animation, comedy, sci-fi",184.0,3.5,2018-09-16 11:49:14,184.0,anime,1.537099e+09
285725,193565,2010,Gintama: The Movie,"action, animation, comedy, sci-fi",184.0,3.5,2018-09-16 11:49:14,184.0,comedy,1.537099e+09
285726,193565,2010,Gintama: The Movie,"action, animation, comedy, sci-fi",184.0,3.5,2018-09-16 11:49:14,184.0,gintama,1.537099e+09


In [29]:
final_movie_rating = final_movie_rating.drop(['userId_y', 'timestamp_y'], axis=1)
final_movie_rating

,movieId,year,title,genres,userId_x,rating,timestamp_x,tag
0,1,1995,Toy Story,"adventure, animation, children, comedy, fantasy",1.0,4.0,2000-07-30 18:45:03,pixar
1,1,1995,Toy Story,"adventure, animation, children, comedy, fantasy",1.0,4.0,2000-07-30 18:45:03,pixar
2,1,1995,Toy Story,"adventure, animation, children, comedy, fantasy",1.0,4.0,2000-07-30 18:45:03,fun
3,1,1995,Toy Story,"adventure, animation, children, comedy, fantasy",5.0,4.0,1996-11-08 06:36:02,pixar
4,1,1995,Toy Story,"adventure, animation, children, comedy, fantasy",5.0,4.0,1996-11-08 06:36:02,pixar
...,...,...,...,...,...,...,...,...
285699,187595,2018,Solo: A Star Wars Story,"action, adventure, children, sci-fi",586.0,5.0,2018-06-25 04:05:56,star wars
285724,193565,2010,Gintama: The Movie,"action, animation, comedy, sci-fi",184.0,3.5,2018-09-16 11:49:14,anime
285725,193565,2010,Gintama: The Movie,"action, animation, comedy, sci-fi",184.0,3.5,2018-09-16 11:49:14,comedy
285726,193565,2010,Gintama: The Movie,"action, animation, comedy, sci-fi",184.0,3.5,2018-09-16 11:49:14,gintama


In [40]:
final_movie_rating['rating'].value_counts()


4.0    64781
5.0    63845
4.5    31499
3.0    28550
3.5    22895
2.0     7955
2.5     6488
1.0     3721
0.5     1908
1.5     1568
Name: rating, dtype: int64

### decide to use movie_rating dataset because tag gives too many duplicates on rating

In [41]:
movie_rating

,movieId,year,title,genres,userId,rating,timestamp
0,1,1995,Toy Story,"adventure, animation, children, comedy, fantasy",1.0,4.0,2000-07-30 18:45:03
1,1,1995,Toy Story,"adventure, animation, children, comedy, fantasy",5.0,4.0,1996-11-08 06:36:02
2,1,1995,Toy Story,"adventure, animation, children, comedy, fantasy",7.0,4.5,2005-01-25 06:52:26
3,1,1995,Toy Story,"adventure, animation, children, comedy, fantasy",15.0,2.5,2017-11-13 12:59:30
4,1,1995,Toy Story,"adventure, animation, children, comedy, fantasy",17.0,4.5,2011-05-18 05:28:03
...,...,...,...,...,...,...,...
100843,193581,2017,Black Butler: Book of the Atlantic,"action, animation, comedy, fantasy",184.0,4.0,2018-09-16 14:44:42
100844,193583,2017,No Game No Life: Zero,"animation, comedy, fantasy",184.0,3.5,2018-09-16 14:52:25
100845,193585,2017,Flint,drama,184.0,3.5,2018-09-16 14:56:45
100846,193587,2018,Bungo Stray Dogs: Dead Apple,"action, animation",184.0,3.5,2018-09-16 15:00:21


In [42]:
movie_rating['rating'].value_counts()


4.0    26813
3.0    20044
5.0    13207
3.5    13132
4.5     8549
2.0     7550
2.5     5548
1.0     2810
1.5     1791
0.5     1369
Name: rating, dtype: int64

stick with movie_rating

## Data Preparation

Describe and justify the process for preparing the data for analysis.

***
Questions to consider:
* Were there variables you dropped or created?
* How did you address missing values or outliers?
* Why are these choices appropriate given the data and the business problem?
***

In [30]:
# Here you run your code to clean the data

## Data Modeling
Describe and justify the process for analyzing or modeling the data.

***
Questions to consider:
* How did you analyze or model the data?
* How did you iterate on your initial approach to make it better?
* Why are these choices appropriate given the data and the business problem?
***

In [31]:
# Here you run your code to model the data


## Evaluation
Evaluate how well your work solves the stated business problem.

***
Questions to consider:
* How do you interpret the results?
* How well does your model fit your data? How much better is this than your baseline model?
* How confident are you that your results would generalize beyond the data you have?
* How confident are you that this model would benefit the business if put into use?
***

## Conclusions
Provide your conclusions about the work you've done, including any limitations or next steps.

***
Questions to consider:
* What would you recommend the business do as a result of this work?
* What are some reasons why your analysis might not fully solve the business problem?
* What else could you do in the future to improve this project?
***